In [4]:
import re
import json

from itertools import chain
import nltk
import pycrfsuite

from ingredient_phrase_tagger.training import utils


In [6]:
ls

__init__.py                  ghg_calculator.ipynb
__pycache__/                 ingredient_phrase_tagger/
app.py                       lib.py
background.png               main_calculation.py
calculator.py                matching.py
data/                        scraper_parser.py
df_recorded_similarities.pk  style.css
df_wiki_similarities.pk      vectorizer.pk


In [8]:
cd ingredient-phrase-tagger-nyt

/Users/georgesdelrieu/code/g-delrieu/git_folder/ingredient-phrase-tagger-nyt


In [7]:
with open('../tests/golden/training_data.crf') as fname:
    lines = fname.readlines()
    items = [line.strip('\n').split('\t') for line in lines]
    items = [item for item in items if len(item)==6]

In [8]:
items[:10]

[['1$1/4', 'I1', 'L20', 'NoCAP', 'NoPAREN', 'B-QTY'],
 ['cups', 'I2', 'L20', 'NoCAP', 'NoPAREN', 'B-UNIT'],
 ['cooked', 'I3', 'L20', 'NoCAP', 'NoPAREN', 'B-COMMENT'],
 ['and', 'I4', 'L20', 'NoCAP', 'NoPAREN', 'I-COMMENT'],
 ['pureed', 'I5', 'L20', 'NoCAP', 'NoPAREN', 'I-COMMENT'],
 ['fresh', 'I6', 'L20', 'NoCAP', 'NoPAREN', 'I-COMMENT'],
 ['butternut', 'I7', 'L20', 'NoCAP', 'NoPAREN', 'B-NAME'],
 ['squash', 'I8', 'L20', 'NoCAP', 'NoPAREN', 'I-NAME'],
 [',', 'I9', 'L20', 'NoCAP', 'NoPAREN', 'OTHER'],
 ['or', 'I10', 'L20', 'NoCAP', 'NoPAREN', 'I-COMMENT']]

In [9]:
sentences = []

sent = [items[0]]
for item in items[1:]:
    if 'I1' in item:
        sentences.append(sent)
        sent = [item]
    else:
        sent.append(item)
len(sentences)

19787

In [10]:

import random
random.shuffle(sentences)
test_size = 0.1
data_size = len(sentences)

test_data = sentences[:int(test_size*data_size)]
train_data = sentences[int(test_size*data_size):]

In [109]:
def sent2labels(sent):
    return [word[-1] for word in sent]

def sent2features(sent):
    return [word[:-1] for word in sent]

def sent2tokens(sent):
    return [word[0] for word in sent]   

y_train = [sent2labels(s) for s in train_data]
X_train = [sent2features(s) for s in train_data]
y_test = [sent2labels(s) for s in test_data]
X_test = [sent2features(s) for s in test_data]
X_train[1]

[['1', 'I1', 'L4', 'NoCAP', 'NoPAREN'],
 ['teaspoon', 'I2', 'L4', 'NoCAP', 'NoPAREN'],
 ['salt', 'I3', 'L4', 'NoCAP', 'NoPAREN']]

In [100]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.43,
    c2=0.012,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)


CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.43, c2=0.012,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [113]:
from sklearn_crfsuite import metrics
labels = list(crf.classes_)

y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.844957884245499

In [101]:
tagger = crf.tagger_

In [85]:
import pickle as cPickle

In [104]:
filename = 'finalized_model.pkl'
cPickle.dump(crf, open(filename, 'wb'))

In [107]:
filename = 'finalized_model.pkl'
loaded_model = cPickle.load(open(filename, 'rb'))
tagger = loaded_model.tagger_

In [15]:
tagger = pycrfsuite.Tagger()
tagger.open('trained_pycrfsuite')

In [114]:
import re
import json
from ingredient_phrase_tagger.training import utils
from string import punctuation

from nltk.tokenize import *

tokenizer = PunktSentenceTokenizer()

filename = 'finalized_model.pkl'
loaded_model = cPickle.load(open(filename, 'rb'))
tagger = loaded_model.tagger_

def get_ingredients_url(url):

    page = requests.get(f'{url}')
    soup = BeautifulSoup(page.content, 'html.parser')
    ingredient = ''

    for a in soup.find_all('li', class_ = "recipe-ingredients__list-item"):
        ingredient += a.get_text()+ '.'
        ingredient += '\n'

    return ingredient
    

def get_sentence_features(sent):
    """Gets  the features of the sentence"""
    sent_tokens = nltk.word_tokenize(utils.cleanUnicodeFractions(sent))

    sent_features = []
    for i, token in enumerate(sent_tokens):
        token_features = [token]
        token_features.extend(utils.getFeatures(token, i+1, sent_tokens))
        sent_features.append(token_features)
    return sent_features

def format_ingredient_output(tagger_output, display=False):
    """Formats the tagger output into a more convenient dictionary"""
    data = [{}]
    display = [[]]
    prevTag = None


    for token, tag in tagger_output:
    # turn B-NAME/123 back into "name"
        tag = re.sub(r'^[BI]\-', "", tag).lower()

        # ---- DISPLAY ----
        # build a structure which groups each token by its tag, so we can
        # rebuild the original display name later.

        if prevTag != tag:
            display[-1].append((tag, [token]))
            prevTag = tag
        else:
            display[-1][-1][1].append(token)
            #               ^- token
            #            ^---- tag
            #        ^-------- ingredient

            # ---- DATA ----
            # build a dict grouping tokens by their tag

            # initialize this attribute if this is the first token of its kind
        if tag not in data[-1]:
            data[-1][tag] = []

        # HACK: If this token is a unit, singularize it so Scoop accepts it.
        if tag == "unit":
            token = utils.singularize(token)

        data[-1][tag].append(token)

    # reassemble the output into a list of dicts.
    output = [
        dict([(k, utils.smartJoin(tokens)) for k, tokens in ingredient.items()])
        for ingredient in data
        if len(ingredient)
    ]

    # Add the raw ingredient phrase
    for i, v in enumerate(output):
        output[i]["input"] = utils.smartJoin(
            [" ".join(tokens) for k, tokens in display[i]])

    return output

def parse_ingredient(sent):
    """ingredient parsing logic"""
    sentence_features = get_sentence_features(sent)
    tags = tagger.tag(sentence_features)
    tagger_output = zip(sent2tokens(sentence_features), tags)
    parsed_ingredient =  format_ingredient_output(tagger_output)
    if parsed_ingredient:
        parsed_ingredient[0]['name'] = parsed_ingredient[0].get('name','').strip('.')
        
    return parsed_ingredient



In [115]:
tagger

In [123]:
def parse_recipe_ingredients(ingredient_list):
    #import pdb; pdb.set_trace()
    """Wrapper around parse_ingredient so we can call it on an ingredient list"""
    sentences = tokenizer.tokenize(ingredient_list)
    sentences = [sent.strip('\n') for sent in sentences]
    names = []
    qtys = []
    units = []
    our_punctuation = '!"#$%&\'())*+:;<=>?@[\\]^_`{|}~'

    for sent in sentences:
        for punctuation in our_punctuation:
        # cleaning for common issues
            sent = sent.replace(punctuation, '')

        sent = sent.replace('can', '')
        sent = sent.replace('package', '')
        sent = sent.replace('container', '')
        sent = sent.replace('eggs eggs', 'eggs')
        sent = sent.replace('⅓', '.33')
        sent = sent.replace('½', '.5')
        sent = sent.replace('¼', '.25')
        sent = sent.replace('¾', '.75')
        sent = sent.replace('tsp', 'teaspoon')
        sent = sent.replace('tbsp', 'tablespoon')
        
        if re.search("\dg", sent) is not None:
            sent = sent.replace("g", "gram", 1)
            
        
        parsed_ingredient = parse_ingredient(sent)
        
        #import pdb; pdb.set_trace()
        if 'name' in parsed_ingredient[0].keys():

            tmp = parsed_ingredient[0]['name']
            useless_quantifiers = ['oz', 'fl', 'ounce']

            try:
                names.append(re.search("[^\d]*$", tmp).group(0))
            except:
                names.append(tmp)


            #else:
            #    names.append(tmp)
        else:
            names.append(np.nan)

        if 'gram' in parsed_ingredient[0]['input']:
            units.append('gram')
        elif 'milliliters' in parsed_ingredient[0]['input']:
            units.append('ml')
        elif 'unit' in parsed_ingredient[0].keys():
            units.append(parsed_ingredient[0]['unit'])
        else:
            units.append('unit')

        if re.search("\dg", sent) is not None:
            try:
                qtys.append(re.search("(\d+)((\d+))+", parsed_ingredient[0]['input']).group(0))
            except:
                pass
        elif 'qty' in parsed_ingredient[0].keys():
            qtys.append(parsed_ingredient[0]['qty'])
        else:
            try:
                qtys.append(float(parsed_ingredient[0]['input'][:3]))
            except:
                qtys.append(np.nan)


    final_df = pd.DataFrame(list(zip(qtys, units, names)), columns = ['qty', 'unit', 'name'])
    final_df = final_df[final_df['name'].notna()]
    final_df = final_df[final_df['unit'].notna()]
    final_df = final_df[final_df['unit'] != 'teaspoon']

    return final_df


def url_to_df(url):
    
    ingredient_list = get_ingredients_url(url)
    
    return parse_recipe_ingredients(ingredient_list)
    

In [124]:
url_to_df('https://www.bbc.co.uk/food/recipes/mango_salmon_39588')

,qty,unit,name
0,520,gram,salmon
5,2,tablespoon,butter
6,1,unit,onion
7,4,clove,garlic
8,0.5,unit,red pepper
9,0.5,unit,carrot
11,4,tablespoon,mango chutney
12,NaN,unit,lemon juice
13,0.5,unit,Scotch bonnet chilli


In [65]:
if re.search("\dg", "2g ground cinnamon").group(0) is not None:
    print("2g ground cinnamon".replace("g", "gram", 1))

2gram ground cinnamon


In [37]:
test_ing = get_ingredients_url('https://www.bbc.co.uk/food/recipes/hummus_chickpea_burgers_94137')


In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
import re
import subprocess
import json
import pandas as pd
import numpy as np

def get_ingredients_url(url):

    page = requests.get(f'{url}')
    soup = BeautifulSoup(page.content, 'html.parser')
    ingredient = ''

    for a in soup.find_all('li', class_ = "recipe-ingredients__list-item"):
        ingredient += a.get_text()+ '.'
        ingredient += '\n'

    return ingredient

In [80]:
test_ing = get_ingredients_url('https://www.bbc.co.uk/food/recipes/hummus_chickpea_burgers_94137')


In [81]:
parse_recipe_ingredients(test_ing)

[{'name': '3-4 tbsp sunflower oil ', 'input': '3-4 tbsp sunflower oil .'},
 {'qty': '1',
  'name': 'onion',
  'other': ',',
  'comment': 'roughly chopped .',
  'input': '1 onion, roughly chopped .'},
 {'qty': '2',
  'name': 'garlic',
  'unit': 'clove',
  'comment': ', thinly sliced .',
  'input': '2 garlic cloves, thinly sliced .'},
 {'qty': '1', 'name': 'tsp ground cumin ', 'input': '1 tsp ground cumin .'},
 {'qty': '1',
  'comment': 'tsp ground',
  'name': 'coriander ',
  'input': '1 tsp ground coriander .'},
 {'comment': '400g tin, drained and rinsed .',
  'name': 'chickpeas',
  'input': '400g tin chickpeas, drained and rinsed .'},
 {'name': '100g/3 1/2oz hummus ', 'input': '100g/3 1/2oz hummus .'},
 {'comment': '50g/2oz plain',
  'name': 'flour ',
  'input': '50g/2oz plain flour .'},
 {'qty': '1/2', 'comment': 'tsp', 'name': 'salt ', 'input': '1/2 tsp salt .'},
 {'comment': 'freshly ground',
  'name': 'black pepper ',
  'input': 'freshly ground black pepper .'},
 {'comment': '50g/2

In [12]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [13]:
trainer.set_params(
{
        'c1': 0.43,
        'c2': 0.012,
        'max_iterations': 100,
        'feature.possible_transitions': True,
        'feature.possible_states': True,
        'linesearch': 'StrongBacktracking'
    }
)

In [99]:
trainer.train('trained_pycrfsuite')